In [58]:
import pandas as pd
train = pd.read_csv('salary-train.csv')



In [59]:
train['FullDescription'] = train['FullDescription'].str.lower()
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=5)
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)
X = vectorizer.fit_transform(train['FullDescription'])
X

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))
#X_test_categ = enc.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_train_categ

<60000x1766 sparse matrix of type '<class 'numpy.float64'>'
	with 120000 stored elements in Compressed Sparse Row format>

In [62]:
from scipy.sparse import hstack
import numpy as np

X_train=hstack([X,X_train_categ])
y=np.array(train['SalaryNormalized'].values)
#X=np.array(X.values)


In [63]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0,random_state=241)
clf.fit(X_train, y)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=241, solver='auto', tol=0.001)

In [64]:
predict = pd.read_csv('salary-test-mini.csv')
predict['FullDescription'] = predict['FullDescription'].str.lower()
predict['FullDescription'] = predict['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
predict['LocationNormalized'].fillna('nan', inplace=True)
predict['ContractTime'].fillna('nan', inplace=True)
X_predict_vect = vectorizer.transform(predict['FullDescription'])
X_predict_categ = enc.transform(predict[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_predict=hstack([X_predict_vect,X_predict_categ])
y_predict=clf.predict(X_predict)


,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,Milton Keynes,contract,NaN
1,a web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [73]:
f1 = open('1.txt', 'w')
f1.write(str(round(y_predict[0],2))+' '+str(round(y_predict[1],2)))
f1.close()
